In [1]:
# 분석시 자주 사용하는 1,2,4번 데이터를 합친 2_4_1.csv 파일을 사용

import pandas as pd
df = pd.read_csv('2_4_1.csv')

In [2]:
# 소수점 2자리 출력을 위한 코드
pd.options.display.float_format = '{:.2f}'.format

# 전체 데이터에서 성별+연령대별 Top5 구매 품목을 선정

In [3]:
from collections import Counter
sex = ['여성','남성']
ages = ['20대','30대','40대','50대','60대','70대']

data = {}
for i in sex:
    df_sex = df.groupby('ma_fem_dv').get_group(i)
    for j in ages:
        x = f'{j}_{i}'
        # 현재는 분석을 위헤 clac_mcls_nm, 즉 중분류를 사용했으나 소분류/대분류 또한 파악 가능
        data[x] = [i[0] for i in Counter(dict(df_sex.groupby('ages').get_group(j)\
                    .groupby('clac_mcls_nm').size())).most_common(5)] # i[0]을 i[1]로 바꾸면 빈도수를 알 수 있음
        
product_type=pd.DataFrame.from_dict(data)
product_type['Top5'] = ['Top_1', 'Top_2', 'Top_3', 'Top_4', 'Top_5']
product_type.set_index('Top5',inplace=True)
product_type

,20대_여성,30대_여성,40대_여성,50대_여성,60대_여성,70대_여성,20대_남성,30대_남성,40대_남성,50대_남성,60대_남성,70대_남성
Top5,,,,,,,,,,,,
Top_1,식당,스낵류,스낵류,스낵류,국산과일,잎채소,스낵류,스낵류,스낵류,스낵류,국산과일,우유
Top_2,스낵류,식당,식당,국산과일,잎채소,두부/콩나물류,일반담배,식당,국산과일,국산과일,잎채소,국산과일
Top_3,비스킷류,비스킷류,국산과일,두부/콩나물류,두부/콩나물류,우유,탄산음료,맥주,식당,맥주,우유,요구르트
Top_4,청소용품,요구르트,비스킷류,잎채소,우유,국산과일,식당,청소용품,요구르트,잎채소,스낵류,잎채소
Top_5,맥주,국산과일,요구르트,식당,식당,요구르트,냉장조리,비스킷류,맥주,우유,두부/콩나물류,수입과일


# 시간대별 구매 TOP 5

In [5]:
import numpy as np
from collections import Counter

top5_time = df.copy()

# 시간대를 나누기 위한 기준 설정
conditions = [\
    (top5_time['de_hr'] >= 4) & (top5_time['de_hr'] <10),\
    (top5_time['de_hr'] >= 10) & (top5_time['de_hr'] <16),\
    (top5_time['de_hr'] >= 16) & (top5_time['de_hr'] <22),\
    (top5_time['de_hr'] >= 22) | (top5_time['de_hr'] <4)\
     ]

# create a list of the values we want to assign for each condition
values = ['아침:4시~9시', '점심:10시~15시', '저녁:16시~21시', '밤:22시~3시']

# create a new column and use np.select to assign values to it using our lists as arguments
top5_time['period'] = np.select(conditions, values)

data = {}
for i in values:
    x = top5_time.groupby('period').get_group(i)
    data[i] = [j for j in Counter(dict(x.groupby('clac_mcls_nm').size())).most_common(5)]
    
top5_time_top5=pd.DataFrame.from_dict(data)
top5_time_top5['Top5'] = ['Top_1', 'Top_2', 'Top_3', 'Top_4', 'Top_5']
top5_time_top5.set_index('Top5',inplace=True)
top5_time_top5

,아침:4시~9시,점심:10시~15시,저녁:16시~21시,밤:22시~3시
Top5,,,,
Top_1,"(일반담배, 5528)","(스낵류, 74396)","(스낵류, 115476)","(스낵류, 10755)"
Top_2,"(냉장조리, 5334)","(식당, 70104)","(국산과일, 63939)","(맥주, 7498)"
Top_3,"(스낵류, 3070)","(국산과일, 48680)","(맥주, 57526)","(우유, 4994)"
Top_4,"(커피음료, 3012)","(잎채소, 45246)","(청소용품, 57045)","(레토르트, 4649)"
Top_5,"(우유, 2967)","(요구르트, 43121)","(요구르트, 56501)","(탄산음료, 4607)"


# + 아래는 위 통계를 성별+연령대별로 추출하는 코드

# 성별_연령대 시간대별 Top 5 품목(중분류)

- 해당 품목이 선택된 횟수를 기준으로 함
(같은 영수증에 중복되게 존재할 수 있음)

In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # windows
# matplotlib.rcParams['font.family'] = 'AppleGothic' # mac
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['axes.unicode_minus'] = False # 한글 폰트 사용 시, 마이너스 글자가 깨지는 현상을 해결
df_offline = df[df.chnl_dv == 1] # 오프라인 관련
df_online = df[df.chnl_dv == 2] # 온라인 관련

def top5(df, age_sex):
    conditions = [\
        (df['de_hr'] >= 4) & (df['de_hr'] <10),\
        (df['de_hr'] >= 10) & (df['de_hr'] <16),\
        (df['de_hr'] >= 16) & (df['de_hr'] <22),\
        (df['de_hr'] >= 22) | (df['de_hr'] <4)\
         ]

    # create a list of the values we want to assign for each condition
    values = ['아침:4시~9시', '점심:10시~15시', '저녁:16시~21시', '밤:22시~3시']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df['period'] = np.select(conditions, values)

    fig, axs = plt.subplots(1, 4, sharey=True, dpi=200)
    fig.subplots_adjust(wspace=0)
    fig.suptitle(age_sex)
    
    for i,j in enumerate(values):
        x = df.groupby('period').get_group(j)
        data = dict(Counter(dict(x.groupby('clac_mcls_nm').size())).most_common(5))    
        clac_mcls_nm = list(data.keys())
        counts = list(data.values())
        axs[i].bar(clac_mcls_nm,counts)
        axs[i].set_title(j, fontsize=5)
        axs[i].set_xticklabels(clac_mcls_nm, fontsize=5, rotation=45)
    
    # 그래프를 이미지로 저장
    plt.savefig(f'{age_sex}.png', dpi=200)
    
sex = ['여성','남성']
ages = ['20대','30대','40대','50대','60대','70대']

for i in sex:
    # 온라인/오프라인/통합 통계는 필요에 따라 맞는 DataFrame을 사용
    df_sex = df_online.groupby('ma_fem_dv').get_group(i)
    for j in ages:
        age_sex = f'{j}_{i}'
        df_age_sex = df_sex.groupby('ages').get_group(j)
        top5(df_age_sex, age_sex)
